# **Measurements**

Measurements operators
- Photon counting
- Homodyne detection of quadratice angle $\phi$
- Heterodyne detection

---
**Note**: All backends support homodyne detection

---

## **Post-selection**

Allowing for specification or post-selection of a required measurement output, and conditioning the remaining unmeasured modes based on this post-selected value.

Post-selection option should be an integer (or list of integers) for MeasureFock, a float for MeasureHomodyne, and a complex value for MeasureHeterodyne.

In [1]:
import strawberryfields as sf
from strawberryfields.ops import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

## **Homodyne measurement**

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\bra}[1]{\langle#1|}$

Gaussian projective measurement given by projecting the state onto the states $\ket{x_\phi}\bra{x_\phi}$

Operations:
- MeasureHomodyne(phi=0) or MeasureX or MeasureP

Backend support:
- "fock"
- "tf"
- "gaussian"

In [55]:
prog = sf.Program(2)
eng = sf.Engine("gaussian")

with prog.context as q:
    S2gate(1) | (q[0], q[1])   # two-mode squeezed gate
    # homodyne detection in the x quadrature of the first iutout mode
    # resultingly, output of mode q[1] is conditionally displaced depending
    # on the measured value
    MeasureHomodyne(0, select=1) | q[0]
    
state = eng.run(prog).state

check displacement of the second output mode we use `reduced_gaussian()` state method to extract the vector of means and the covariance matrix:

In [48]:
mu, cov = state.reduced_gaussian([1])

The vector of means contains the mean quadrature displacements and for a single mode if of the form $\vec{r}=(\vec{x}, \vec{p})$.

Therefore looking at the first index of the vector of means for $q[1]$

In [49]:
print( "Mu: ", mu[0] )

Mu:  0.9640275698261901


The $x$ quadrature displacement of the second mode is conditional to the post-selected value in the circuit construction.

## **Heterodyne measurement**

The heterodyne measurement can only be performed in the Gaussian backend.
$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\bra}[1]{\langle#1|}$

Heterodyne measurement is a Gaussian projective measurement given by projecting the state onto the coherent states, $\frac{1}{\pi}\ket{\alpha}\bra{\alpha}$

Operations:
- MeasureHeterodyne()

Backend support:
- "gaussian"

## **Photon counting measurement**

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\bra}[1]{\langle#1|}$

A non-Gaussian projective measurement given by $\ket{n_i}\bra{n_i}$

---
Warning: Photon counting is available in the Gaussian backend, but the state of the circuit is not updated after measurement (since it would be non-Gaussian).

---

Operations:
- MeasureFock()

Backend support:
- "fock"
- "tf"
- "gaussian"

In [51]:
# set the random seed
np.random.seed(42)

prog = sf.Program(2)
eng = sf.Engine("fock", backend_options={"cutoff_dim": 6})

with prog.context as q:
    Fock(2) | q[0]
    Fock(3) | q[1]
    BSgate() | (q[0], q[1])    # 50-50 beam splitter implied BSgate(np.pi/4, 0.0)
    MeasureFock(select=0) | q[0] # post-selection
    MeasureFock() | q[1]
    
results = eng.run(prog)

# extract the measured value of mode q[0] via the results object returned by the engine after it has finished execution:
print( "Results q[0]: ", results.samples[0][0])
print( "Results q[1]: ", results.samples[0][1] )

Results q[0]:  0
Results q[1]:  5
